In [ ]:
#import packages
from skimage.io import imread
from skimage.transform import rescale
from shapely.geometry import MultiPolygon, Polygon
from collections import defaultdict
import geopandas
import pandas as pd
import cv2
import numpy as np
import json
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#convert image_mask to polygons
#code origin: https://michhar.github.io/masks_to_polygons_and_back/

def mask_to_polygons(mask, epsilon=10., min_area=10.):
    """Convert a mask ndarray (binarized image) to Multipolygons"""
       
    # first, find contours with cv2: it's much faster than shapely
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    all_polygons = MultiPolygon(all_polygons)

    return all_polygons

In [ ]:
#load reference_list

sample_path = "./demo_brain/"

rsplist = pd.read_csv (sample_path + "reference_list.csv", index_col = 0)
rsplist["mask_number"]= [1320-int(a) for a in rsplist["0"].tolist()]
rsplist

In [ ]:
#generate cfos_spot_csv/slice from Imaris spot_position.csv 
cfos_spot = pd.read_csv (sample_path + "position_Detailed.csv")

cfos_spot_folder = sample_path + "cfos_spot"
if not os.path.exists(cfos_spot_folder):
    os.makedirs(cfos_spot_folder)

for b in np.arange (len(rsplist))+1:
    spot =[]
    for indexc, varac in cfos_spot [cfos_spot["Time"]==b].iterrows():
        #"/1.76" means convert coordinate from 1.76 micrometer/pixel to 1 pixel/pixel
        point =(cfos_spot["Position X"][indexc]/1.76, cfos_spot["Position Y"][indexc]/1.76) 
        spot.append (point)
    pd.DataFrame (spot, columns = ["0", "1"]).to_csv (cfos_spot_folder + "/spot"+str (b)+".csv")

In [ ]:
#batch analysis of oChIEF_b and C-FOS signals

#Brain region list for analysis
structure_list = pd.read_csv("./CCFv3/structuretree/structure_list_paper.csv", index_col = 0)

#subset brain regions
DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

structure_list_idx = structure_list[structure_list["LABEL"].isin(DMN_list)].IDX.tolist()

for c in structure_list_idx:
    
    structrue_index = structure_list[structure_list["IDX"]==c].index.values[0]
    slice_number = []    
    cfos_spot_L = []
    cfos_spot_R = []
    spot_L_size = []
    spot_R_size =[]
    area_all =[]
    cfos_L_densitylist =[]
    cfos_R_densitylist = []    
    oChIEF_L_binary_list = []
    oChIEF_R_binary_list = []
    
    structure_path = sample_path +"quantification/"+str(structure_list.loc[structrue_index, "LABEL"])
    
    if not os.path.exists(structure_path):
        os.makedirs(structure_path)


    for d in rsplist["mask_number"]:
        
        if  d in json.loads(structure_list["section_number"][structrue_index]):
            
            slice_n = rsplist["mask_number"].to_list().index(d)+1
            slice_number.append (slice_n)
            
            #open matched regional mask within our experiment slices
            #regional masks can be obtained from Allen Brain Reference Space
            #https://allensdk.readthedocs.io/en/latest/_static/examples/nb/reference_space.html
            mask=pd.read_csv ("./CCFv3/mask/mask"+str(structure_list["IDX"][structrue_index])+"/mask"+str(d)+".csv", header = 0, index_col = 0).values
        
            # enlarge to highcontent image scale (1140 x 800 pixels to 6480 x 4547 pixels)
            highcontent_mask = rescale (mask, 5.6842105263, anti_aliasing=False)
            
            #convert to 8 bit binary image mask
            highcontent_mask_bw =(highcontent_mask>np.mean (highcontent_mask)).astype("uint8")
        

            # convert 8 bit binary image mask to shapely multipolygon 
            mask_poly = mask_to_polygons(highcontent_mask_bw, min_area = 10)

            #write multipolygon into geoseries
            mask_geopd = geopandas.GeoSeries([mask_poly])
        
            #open matched cfos spot csv
            spot = pd.read_csv (sample_path +"cfos_spot/spot"+str(rsplist["mask_number"].to_list().index(d)+1)+".csv", header = 0, index_col = 0)     
            spotL = spot[spot["0"]<3240]
            spotR = spot[spot["0"]>=3240]

            #convert cfos spot DataFrame to GeoDataFrame containing shapely points
            spotL_geopd = geopandas.GeoDataFrame (spotL, geometry = geopandas.points_from_xy(spotL["0"], spotL["1"])) 
            spotR_geopd = geopandas.GeoDataFrame (spotR, geometry = geopandas.points_from_xy(spotR["0"], spotR["1"]))
            
            #calculate cfos density within regional mask (unit: #/ square minimeter)
            cfosL_density = float(len(spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))])/mask_geopd.area)/(1.76**2)*2000000
            cfos_L_densitylist.append(cfosL_density)       
            cfosL_spot = np.transpose([spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]["0"].tolist(), 
                          spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]["1"].tolist(),
                          (np.zeros (len (spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]["0"]))+slice_n).tolist()]).tolist()        
            cfos_spot_L+=cfosL_spot
            spot_L_size.append (len(spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]))
        
            cfosR_density = float(len(spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))])/mask_geopd.area)/(1.76**2)*2000000
            cfos_R_densitylist.append(cfosR_density)
            cfosR_spot = np.transpose([spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]["0"].tolist(), 
                          spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]["1"].tolist(),
                          (np.zeros (len (spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]["0"]))+slice_n).tolist()]).tolist()
            cfos_spot_R+=cfosR_spot
            spot_R_size.append (len(spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]))
            
            area_all.append (float(mask_geopd.area))
            
            #open oChIEF binary image size: 6480 x 4547 pixels
            binary_img = imread(sample_path + "oChIEF_T_b/oChIEF_T_b"+"%04d"%(rsplist["mask_number"].to_list().index(d))+".png")/255
            
            #only show mask region oChIEF signals
            binary_imgmask = np.where (highcontent_mask>np.mean(highcontent_mask), binary_img, 0)
            L_b = binary_imgmask[:, :3240]
            R_b = binary_imgmask[:,3240:]
            
            #calculate regional oChIEF mean pixels
            mean_L_b = (np.mean (L_b))
            oChIEF_L_binary_list.append(mean_L_b)
                   
            mean_R_b = (np.mean (R_b))
            oChIEF_R_binary_list.append(mean_R_b)
            
    pd.DataFrame (list(zip(slice_number, cfos_L_densitylist, cfos_R_densitylist)),columns=["slice_number","cfos_L_density", "cfos_R_density"]).to_csv (structure_path +"/cfos_density.csv")
    pd.DataFrame ({"slice_number":slice_number, "cfos_L_spot_number": spot_L_size, "cfos_R_spot_number": spot_R_size, "structure_area (pixels)": area_all}).to_csv (structure_path +"/cfos_density_raw.csv")
    pd.DataFrame (list(zip(slice_number, oChIEF_L_binary_list, oChIEF_R_binary_list)),columns=["slice_number","oChIEF_L_b_mean", "oChIEF_R_b_mean"]).to_csv (structure_path +"/oChIEF_meanint_b.csv")
    pd.DataFrame (cfos_spot_L, columns = ["x(pixels)","y(pixels)","z(pixels)"]).to_csv (structure_path +"/cfos_L_spot_location.csv")
    pd.DataFrame (cfos_spot_R, columns = ["x(pixels)","y(pixels)","z(pixels)"]).to_csv (structure_path +"/cfos_R_spot_location.csv")

In [ ]:
#summary of oChIEF_data_binarizedimages
quantification_path = sample_path +"quantification/"

oChIEF_L_summary = pd.DataFrame (index = DMN_list, columns = np.arange (len(rsplist))+1)
oChIEF_R_summary = pd.DataFrame (index = DMN_list, columns = np.arange (len(rsplist))+1)

for f in DMN_list:
    for root, dirs, files in os.walk (quantification_path +f):
        for filef in files:
            if filef.endswith ("oChIEF_meanint_b.csv"):
                oChIEF = pd.read_csv (os.path.join(root, filef), index_col = 0)
                for indexe, varae in oChIEF.iterrows():
                    oChIEF_L_summary.at [f, oChIEF.slice_number[indexe]]=oChIEF.oChIEF_L_b_mean[indexe]
                    oChIEF_R_summary.at [f, oChIEF.slice_number[indexe]]=oChIEF.oChIEF_R_b_mean[indexe]
                    
oChIEF_L_summary.to_csv (sample_path + "oChIEF_L_summary_b.csv")
oChIEF_R_summary.to_csv (sample_path +"oChIEF_R_summary_b.csv")

In [ ]:
#summary of cfos_density_data
quantification_path = sample_path +"quantification/"

cfos_L_summary = pd.DataFrame (index = DMN_list, columns = np.arange (len(rsplist))+1)
cfos_R_summary = pd.DataFrame (index = DMN_list, columns = np.arange (len(rsplist))+1)

for f in DMN_list:
    for root, dirs, files in os.walk (quantification_path +f):
            for file in files:
                if file.endswith ("cfos_density.csv"):
                    cfos = pd.read_csv (os.path.join(root, file), index_col = 0)
                    for indexa, varaa in cfos.iterrows():
                        cfos_L_summary.at [f, cfos.slice_number[indexa]]=cfos.cfos_L_density[indexa]
                        cfos_R_summary.at [f, cfos.slice_number[indexa]]=cfos.cfos_R_density[indexa]
cfos_L_summary.to_csv (sample_path + "cfos_L_summary.csv")
cfos_R_summary.to_csv (sample_path + "cfos_R_summary.csv")

In [ ]:
#summary of cfos_number_data
quantification_path = sample_path +"quantification/"

cfos_L_summary = pd.DataFrame (index = DMN_list, columns = np.arange (len(rsplist))+1)
cfos_R_summary = pd.DataFrame (index = DMN_list, columns = np.arange (len(rsplist))+1)

for f in DMN_list:
    for root, dirs, files in os.walk (quantification_path +f):
            for file in files:
                if file.endswith ("cfos_density_raw.csv"):
                    cfos = pd.read_csv (os.path.join(root, file), index_col = 0)
                    for indexa, varaa in cfos.iterrows():
                        cfos_L_summary.at [f, cfos.slice_number[indexa]]=cfos.cfos_L_spot_number[indexa]
                        cfos_R_summary.at [f, cfos.slice_number[indexa]]=cfos.cfos_R_spot_number[indexa]
cfos_L_summary.to_csv (sample_path + "cfos_L_num_summary.csv")
cfos_R_summary.to_csv (sample_path + "cfos_R_num_summary.csv")

In [ ]:
#summary of area_data
quantification_path = sample_path +"quantification/"

area_summary = pd.DataFrame (index = DMN_list, columns = np.arange (len(rsplist))+1)

for f in DMN_list:
    for root, dirs, files in os.walk (quantification_path +f):
            for file in files:
                if file.endswith ("cfos_density_raw.csv"):
                    area = pd.read_csv (os.path.join(root, file), index_col = 0)
                    for indexa, varaa in area.iterrows():
                        area_summary.at [f, area.slice_number[indexa]]=(area["structure_area (pixels)"][indexa])/(2*(1.76**2))
area_summary.to_csv (sample_path + "area_summary.csv")    

put the quantified data to folder "./quantification/genotype/st_pattern/Sample_folder"